In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pymongo csetutils aiohttp crm_notate

In [ ]:
from pymongo import MongoClient

mongo_client = MongoClient('')
db = mongo_client['phishstory']
collection = db['incidents']

In [ ]:
import json

close_reasons = ['shopper_compromise', 'intentionally_malicious']
shoppers = dict()
for reason in close_reasons:
    domain_shoppers = collection.aggregate([
        { '$match': {'phishstory_status': 'CLOSED', 'close_reason': reason}},
        { '$group': { '_id': { 'shopper': '$data.domainQuery.shopperInfo.shopperId' } } }
    ])
    host_shoppers = collection.aggregate([
        { '$match': {'phishstory_status': 'CLOSED', 'close_reason': reason}},
        { '$group': { '_id': { 'shopper': '$data.domainQuery.host.shopperId' } } }
    ])
    i = 0
    for ticket in domain_shoppers:
        shoppers[ticket['_id']['shopper']] = reason
    for ticket in host_shoppers:
        shoppers[ticket['_id']['shopper']] = reason

with open(f'shoppers.json', 'w') as out:
    out.write(json.dumps(shoppers))


In [ ]:
import json
import requests

with open(f'shoppers.json', 'r') as out:
    shoppers = json.loads(out.read())

cert =  ('test.crt', 'test.key')
response = requests.post('https://sso.gdcorp.tools/v1/secure/api/token', data={'realm': 'cert'}, cert=cert)
response.raise_for_status()
body = response.json()
jwt = body.get('data')

shoppers_needing_locks = {}
headers = {'Content-Type': 'application/json', 'Authorization': f'sso-jwt {jwt}'}
i = 0
total = len(shoppers)
for shopper, reason in shoppers.items():
    i += 1
    r = requests.get(f'https://shoplocked.cset.int.gdcorp.tools/status/{shopper}', headers=headers)
    if r.status_code in [401, 403]:
        response = requests.post('https://sso.gdcorp.tools/v1/secure/api/token', data={'realm': 'cert'}, cert=cert)
        response.raise_for_status()
        body = response.json()
        jwt = body.get('data')
        headers = {'Content-Type': 'application/json', 'Authorization': f'sso-jwt {jwt}'}
        r = requests.get(f'https://shoplocked.cset.int.gdcorp.tools/status/{shopper}', headers=headers)
    res = r.json()
    if 'message' in res and res['message'] == f'Unable to get lock status, Invalid shopper ID {shopper}':
        pass
    elif 'admin_lock' in res and res['admin_lock']:
        shoppers_needing_locks[shopper] = reason
    elif 'admin_lock' in res and not res['admin_lock']:
        pass
    else:
        print(res)
    if i % 100 == 0:
        print(i)
print(len(shoppers_needing_locks))
with open(f'shoppers-filtered.json', 'w') as out:
    out.write(json.dumps(shoppers_needing_locks))

In [ ]:
blocklisted_domains = []
with open(f'shoppers-filtered.json', 'r') as out:
    shoppers = json.loads(out.read())
post_filter = {}
print(len(shoppers))
for shopper, reason in shoppers.items():
    if shopper not in blocklisted_domains:
        post_filter[shopper] = reason
print(len(post_filter))
with open(f'shoppers-post-blocklist.json', 'w') as out:
    out.write(json.dumps(post_filter))

In [ ]:
import json
from csetutils.services.sso import SSOClient, LockTeamIDs, LockType, Causes


with open(f'shoppers-post-blocklist.json', 'r') as out:
    shoppers = json.loads(out.read())

client = SSOClient(
    'https://sso.gdcorp.tools',
    'test.crt',
    'test.key', 
    'prod'
)

i = 0
total = len(shoppers)
for shopper, reason in shoppers.items():
    i += 1
    if reason == 'intentionally_malicious':
        lock_type = LockType.adminTerminated
        cause = Causes.Policy
        team = LockTeamIDs.LtSecurity
    else:
        lock_type = LockType.adminCompromised
        cause = Causes.Compromise
        team = LockTeamIDs.LtCompromise

    client.lock_idp(int(shopper), lock_type, cause, team, 'migrating shopper lock to new admin lock system', 'nwade')
    if i % 200 == 0:
        print(f'{i}/{total}')

In [ ]:
import json
import requests

with open(f'shoppers-post-blocklist.json', 'r') as out:
    shoppers = json.loads(out.read())

cert =  ('test.crt', 'test.key')
response = requests.post('https://sso.gdcorp.tools/v1/secure/api/token', data={'realm': 'cert'}, cert=cert)
response.raise_for_status()
body = response.json()
jwt = body.get('data')
headers = {'Content-Type': 'application/json', 'Authorization': f'sso-jwt {jwt}'}
shoppers = [key for key, value in shoppers.items()]
total = len(shoppers)
for i in range(0, len(shoppers), 100):
    payload = {
        'creds': shoppers[i:i+100],
        'note': 'migrating legacy shopper locks to new admin lock system',
        'products': ['crm']
    }
    r = requests.post('https://shoplocked.cset.int.gdcorp.tools/unlockAdmin', json=payload, headers=headers, timeout=120)
    if r.status_code in [401,  403]:
        response = requests.post('https://sso.gdcorp.tools/v1/secure/api/token', data={'realm': 'cert'}, cert=cert)
        response.raise_for_status()
        body = response.json()
        jwt = body.get('data')
        headers = {'Content-Type': 'application/json', 'Authorization': f'sso-jwt {jwt}'}
        r = requests.post('https://shoplocked.cset.int.gdcorp.tools/unlockAdmin', json=payload, headers=headers, timeout=120)
    r.raise_for_status()
    if i % 200 == 0:
        print(f'{i}/{total}')
        